In [18]:
!pip uninstall pyarrow -y

Found existing installation: pyarrow 15.0.0
Uninstalling pyarrow-15.0.0:
  Successfully uninstalled pyarrow-15.0.0


In [19]:
pip install pyarrow==12.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 18.2 MB/s eta 0:00:00


In [1]:
!pip install --upgrade datasets

In [15]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [2]:
import pyarrow
print(pyarrow.__version__)

12.0.0


In [3]:
import datasets
print(datasets.__version__)

2.17.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ds = 'manual' # 'mrpc','manual'
sentiment = 'positive' # 'positive', 'negative'

folder_input_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/corpora/'
folder_pretrained_path = '/content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/'
csv_file_path = f'{ds}-triplet-corpus.csv'

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
from datasets import load_dataset
import torch

In [7]:
# Load the tokenizer and model
model_checkpoint = 'prithivida/parrot_paraphraser_on_T5'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
import pandas as pd
# Load and prepare dataset
df = pd.read_csv(folder_input_path + csv_file_path)
if sentiment == 'positive':
    dataset = df[['original', 'positive']].rename(columns={'original': 'input_text', 'positive': 'target_text'})
else:
    dataset = df[['original', 'negative']].rename(columns={'original': 'input_text', 'negative': 'target_text'})

dataset = Dataset.from_pandas(dataset)

In [11]:
def preprocess_function(examples):
    # Tokenize the input texts
    model_inputs = tokenizer(examples["input_text"], max_length=128, truncation=True)

    # Tokenize the target texts in a separate call using `text_target` argument
    labels = tokenizer(text_target=examples["target_text"], max_length=128, truncation=True)

    # Assign the tokenized target texts' input IDs as the labels for the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

In [16]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=folder_pretrained_path,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model and tokenizer
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`